## Import & Install

In [ ]:
#!pip install -r requirements.txt

In [ ]:
import pandas as pd
import random
import os
import numpy as np
import warnings
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from datetime import datetime

from sklearn.preprocessing import RobustScaler, PowerTransformer

from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Load

In [ ]:
###  INPUT ###
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

## Data Pre-processing

In [ ]:
input_data = input_data.drop(columns=['frmDist'])

# Split the data into training and testing sets
X = input_data[input_data.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = input_data[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]

In [ ]:
X.columns = X.columns.str.replace(' ', '_')

In [ ]:
value_counts_per_column = X.nunique()
multi_value_columns = value_counts_per_column[value_counts_per_column > 1].index
X = X[multi_value_columns]

In [ ]:
X = X.drop(columns=['frmYear', 'frmWeek'])

## Scaling

In [ ]:
num_features = X.columns[X.columns != 'date']

In [ ]:
scaler = PowerTransformer()
X[num_features] = scaler.fit_transform(X[num_features])

## Feature Selection

In [ ]:
X_1 = X.drop(columns=['tcdmt',
                      'WaterUsage',
                      'WaterCost',
                      'FertilizerUsage',
                      'FertilizerCost',
                      'CO2Usage',
                      'CO2Cost',
                      'MistUsageTime',
                      'Mist_Cost'])

In [ ]:
X_2 = X.drop(columns=['hvstCo'])

## Add time variable

In [ ]:
date = pd.to_datetime(X_1['date'], format='%Y%m%d')
X_1.loc[:, 'year'] = date.dt.year
X_1.loc[:, 'month'] = date.dt.month
X_1.loc[:, 'week'] = date.dt.isocalendar().week.astype(np.int32)
X_1.loc[:, 'day'] = date.dt.weekday

In [ ]:
date = pd.to_datetime(X_2['date'], format='%Y%m%d')
X_2.loc[:, 'year'] = date.dt.year
X_2.loc[:, 'month'] = date.dt.month
X_2.loc[:, 'week'] = date.dt.isocalendar().week.astype(np.int32)
X_2.loc[:, 'day'] = date.dt.weekday

In [ ]:
X_1 = X_1.drop(columns='date')
X_2 = X_2.drop(columns='date')

## Train_test_split

In [ ]:
train_01, test_01, train_y_01, test_y_01 = train_test_split(X_1, Y['outtrn_cumsum'], test_size=0.2, random_state=42)
train_02, test_02, train_y_02, test_y_02 = train_test_split(X_2, Y['HeatingEnergyUsage_cumsum'], test_size=0.2, random_state=42)

In [ ]:
train_01 = train_01.reset_index(drop=True)
test_01 = test_01.reset_index(drop=True)
train_y_01 = train_y_01.reset_index(drop=True)
test_y_01 = test_y_01.reset_index(drop=True)

In [ ]:
train_02 = train_02.reset_index(drop=True)
test_02 = test_02.reset_index(drop=True)
train_y_02 = train_y_02.reset_index(drop=True)
test_y_02 = test_y_02.reset_index(drop=True)

## Modeling

In [ ]:
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [ ]:
# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    """
    Calculate the Root Mean Squared Error (RMSE) between predicted and target values.

    :param predictions: Predicted values.
    :type predictions: array-like
    :param targets: Target values.
    :type targets: array-like
    :return: RMSE value.
    :rtype: float
    """
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    from sklearn.metrics import r2_score
    return r2_score(y_test, y_pred)

In [ ]:
X_train_01, X_val_01, y_train_01, y_val_01 = train_test_split(train_01, train_y_01, test_size=0.2, random_state=42)

In [ ]:
params = {
    'iterations' : 100000,
    'learning_rate' : 0.05,
    'loss_function' : 'RMSE',
    'eval_metric': 'RMSE',
    'early_stopping_rounds': 50,
    'task_type': 'GPU',
    'random_state': 42
}

model = CatBoostRegressor(**params)

model.fit(X_train_01, y_train_01, eval_set=(X_val_01, y_val_01))

y_pred_01 = model.predict(test_01)

0:	learn: 37001.6229915	test: 36643.6434291	best: 36643.6434291 (0)	total: 3.35ms	remaining: 5m 35s
1:	learn: 35361.1973639	test: 35019.0166972	best: 35019.0166972 (1)	total: 6.49ms	remaining: 5m 24s
2:	learn: 33818.2118887	test: 33487.4276451	best: 33487.4276451 (2)	total: 8.82ms	remaining: 4m 53s
3:	learn: 32333.7374247	test: 32013.9321927	best: 32013.9321927 (3)	total: 11.4ms	remaining: 4m 44s
4:	learn: 30936.9824536	test: 30623.6456937	best: 30623.6456937 (4)	total: 14.5ms	remaining: 4m 49s
5:	learn: 29623.8723960	test: 29307.9141778	best: 29307.9141778 (5)	total: 16.7ms	remaining: 4m 38s
6:	learn: 28388.6870022	test: 28079.1616363	best: 28079.1616363 (6)	total: 19ms	remaining: 4m 32s
7:	learn: 27222.3602039	test: 26921.7480910	best: 26921.7480910 (7)	total: 21.4ms	remaining: 4m 27s
8:	learn: 26108.2425813	test: 25804.3255474	best: 25804.3255474 (8)	total: 23.7ms	remaining: 4m 23s
9:	learn: 25041.8801621	test: 24736.0118336	best: 24736.0118336 (9)	total: 27ms	remaining: 4m 29s
10:	

83:	learn: 5503.1948874	test: 5370.1721283	best: 5370.1721283 (83)	total: 220ms	remaining: 4m 21s
84:	learn: 5455.8668703	test: 5327.6408911	best: 5327.6408911 (84)	total: 223ms	remaining: 4m 21s
85:	learn: 5401.0496833	test: 5274.4485505	best: 5274.4485505 (85)	total: 225ms	remaining: 4m 21s
86:	learn: 5359.6483090	test: 5235.8512646	best: 5235.8512646 (86)	total: 227ms	remaining: 4m 20s
87:	learn: 5311.7764040	test: 5189.8169633	best: 5189.8169633 (87)	total: 229ms	remaining: 4m 20s
88:	learn: 5245.8116135	test: 5131.2737465	best: 5131.2737465 (88)	total: 232ms	remaining: 4m 20s
89:	learn: 5208.1105813	test: 5098.8424137	best: 5098.8424137 (89)	total: 236ms	remaining: 4m 21s
90:	learn: 5163.9506983	test: 5056.6798340	best: 5056.6798340 (90)	total: 238ms	remaining: 4m 21s
91:	learn: 5124.7293626	test: 5022.4908850	best: 5022.4908850 (91)	total: 240ms	remaining: 4m 20s
92:	learn: 5089.8761612	test: 4988.6664263	best: 4988.6664263 (92)	total: 243ms	remaining: 4m 20s
93:	learn: 5040.3297

166:	learn: 3396.8270099	test: 3386.9205402	best: 3386.9205402 (166)	total: 425ms	remaining: 4m 14s
167:	learn: 3388.5966801	test: 3376.9110614	best: 3376.9110614 (167)	total: 427ms	remaining: 4m 13s
168:	learn: 3381.5949031	test: 3370.1464745	best: 3370.1464745 (168)	total: 429ms	remaining: 4m 13s
169:	learn: 3373.6716000	test: 3363.1050601	best: 3363.1050601 (169)	total: 432ms	remaining: 4m 13s
170:	learn: 3366.0597309	test: 3356.4534274	best: 3356.4534274 (170)	total: 434ms	remaining: 4m 13s
171:	learn: 3356.2602622	test: 3346.5582122	best: 3346.5582122 (171)	total: 436ms	remaining: 4m 13s
172:	learn: 3345.3103749	test: 3335.6107338	best: 3335.6107338 (172)	total: 439ms	remaining: 4m 13s
173:	learn: 3335.4463270	test: 3326.2439112	best: 3326.2439112 (173)	total: 442ms	remaining: 4m 13s
174:	learn: 3317.6865131	test: 3311.6119522	best: 3311.6119522 (174)	total: 444ms	remaining: 4m 13s
175:	learn: 3310.4543713	test: 3305.3996855	best: 3305.3996855 (175)	total: 447ms	remaining: 4m 13s


248:	learn: 2653.4237795	test: 2673.6154762	best: 2673.6154762 (248)	total: 632ms	remaining: 4m 13s
249:	learn: 2647.8874509	test: 2668.2778579	best: 2668.2778579 (249)	total: 634ms	remaining: 4m 13s
250:	learn: 2635.2897459	test: 2658.5326211	best: 2658.5326211 (250)	total: 637ms	remaining: 4m 13s
251:	learn: 2630.5530925	test: 2653.5882511	best: 2653.5882511 (251)	total: 640ms	remaining: 4m 13s
252:	learn: 2625.5974747	test: 2648.7573397	best: 2648.7573397 (252)	total: 642ms	remaining: 4m 13s
253:	learn: 2612.3299508	test: 2636.5690685	best: 2636.5690685 (253)	total: 645ms	remaining: 4m 13s
254:	learn: 2608.6804338	test: 2633.1409395	best: 2633.1409395 (254)	total: 648ms	remaining: 4m 13s
255:	learn: 2597.9880178	test: 2622.9299817	best: 2622.9299817 (255)	total: 651ms	remaining: 4m 13s
256:	learn: 2593.5637063	test: 2618.3251937	best: 2618.3251937 (256)	total: 653ms	remaining: 4m 13s
257:	learn: 2581.7283973	test: 2607.3629122	best: 2607.3629122 (257)	total: 655ms	remaining: 4m 13s


330:	learn: 2236.0749680	test: 2274.4763948	best: 2274.4763948 (330)	total: 847ms	remaining: 4m 14s
331:	learn: 2231.4911035	test: 2269.6344721	best: 2269.6344721 (331)	total: 850ms	remaining: 4m 15s
332:	learn: 2227.5895663	test: 2265.4702130	best: 2265.4702130 (332)	total: 852ms	remaining: 4m 15s
333:	learn: 2224.6940700	test: 2262.1698331	best: 2262.1698331 (333)	total: 855ms	remaining: 4m 15s
334:	learn: 2222.4914097	test: 2259.6608736	best: 2259.6608736 (334)	total: 858ms	remaining: 4m 15s
335:	learn: 2220.8671420	test: 2257.6575790	best: 2257.6575790 (335)	total: 861ms	remaining: 4m 15s
336:	learn: 2216.8746328	test: 2253.6305183	best: 2253.6305183 (336)	total: 863ms	remaining: 4m 15s
337:	learn: 2214.4384179	test: 2251.5516380	best: 2251.5516380 (337)	total: 866ms	remaining: 4m 15s
338:	learn: 2209.5381960	test: 2246.5580359	best: 2246.5580359 (338)	total: 869ms	remaining: 4m 15s
339:	learn: 2207.5784925	test: 2244.8510019	best: 2244.8510019 (339)	total: 871ms	remaining: 4m 15s


...

3761:	learn: 1190.5038569	test: 1546.8290581	best: 1546.8093565 (3749)	total: 9.17s	remaining: 3m 54s
3762:	learn: 1190.4076641	test: 1546.7655146	best: 1546.7655146 (3762)	total: 9.17s	remaining: 3m 54s
3763:	learn: 1190.3202727	test: 1546.7704402	best: 1546.7655146 (3762)	total: 9.17s	remaining: 3m 54s
3764:	learn: 1190.2630434	test: 1546.7934584	best: 1546.7655146 (3762)	total: 9.18s	remaining: 3m 54s
3765:	learn: 1190.1843248	test: 1546.7601989	best: 1546.7601989 (3765)	total: 9.18s	remaining: 3m 54s
3766:	learn: 1190.0707378	test: 1546.7088938	best: 1546.7088938 (3766)	total: 9.18s	remaining: 3m 54s
3767:	learn: 1189.9088957	test: 1546.6113022	best: 1546.6113022 (3767)	total: 9.18s	remaining: 3m 54s
3768:	learn: 1189.8131620	test: 1546.6767541	best: 1546.6113022 (3767)	total: 9.18s	remaining: 3m 54s
3769:	learn: 1189.7453195	test: 1546.6650491	best: 1546.6113022 (3767)	total: 9.19s	remaining: 3m 54s
3770:	learn: 1189.6952278	test: 1546.6315429	best: 1546.6113022 (3767)	total: 9.19

3842:	learn: 1183.5110163	test: 1545.2942134	best: 1545.2559912 (3840)	total: 9.36s	remaining: 3m 54s
3843:	learn: 1183.3532494	test: 1545.1790069	best: 1545.1790069 (3843)	total: 9.37s	remaining: 3m 54s
3844:	learn: 1183.2951094	test: 1545.1728558	best: 1545.1728558 (3844)	total: 9.37s	remaining: 3m 54s
3845:	learn: 1183.2175211	test: 1545.1351188	best: 1545.1351188 (3845)	total: 9.37s	remaining: 3m 54s
3846:	learn: 1183.1743577	test: 1545.1933106	best: 1545.1351188 (3845)	total: 9.37s	remaining: 3m 54s
3847:	learn: 1183.1215648	test: 1545.1608464	best: 1545.1351188 (3845)	total: 9.38s	remaining: 3m 54s
3848:	learn: 1183.0801832	test: 1545.1765172	best: 1545.1351188 (3845)	total: 9.38s	remaining: 3m 54s
3849:	learn: 1182.8977427	test: 1545.0292256	best: 1545.0292256 (3849)	total: 9.38s	remaining: 3m 54s
3850:	learn: 1182.8490191	test: 1544.9970998	best: 1544.9970998 (3850)	total: 9.38s	remaining: 3m 54s
3851:	learn: 1182.7585180	test: 1544.9372405	best: 1544.9372405 (3851)	total: 9.39

3923:	learn: 1176.9223612	test: 1543.6903126	best: 1543.6485811 (3922)	total: 9.55s	remaining: 3m 53s
3924:	learn: 1176.8599934	test: 1543.6977889	best: 1543.6485811 (3922)	total: 9.56s	remaining: 3m 53s
3925:	learn: 1176.8132634	test: 1543.7345350	best: 1543.6485811 (3922)	total: 9.56s	remaining: 3m 53s
3926:	learn: 1176.7242214	test: 1543.8015747	best: 1543.6485811 (3922)	total: 9.56s	remaining: 3m 53s
3927:	learn: 1176.6318387	test: 1543.7534940	best: 1543.6485811 (3922)	total: 9.56s	remaining: 3m 53s
3928:	learn: 1176.5853562	test: 1543.7578428	best: 1543.6485811 (3922)	total: 9.57s	remaining: 3m 53s
3929:	learn: 1176.4195427	test: 1543.8163797	best: 1543.6485811 (3922)	total: 9.57s	remaining: 3m 53s
3930:	learn: 1176.3634968	test: 1543.8166729	best: 1543.6485811 (3922)	total: 9.57s	remaining: 3m 53s
3931:	learn: 1176.2916721	test: 1543.7461645	best: 1543.6485811 (3922)	total: 9.57s	remaining: 3m 53s
3932:	learn: 1176.2567842	test: 1543.7005254	best: 1543.6485811 (3922)	total: 9.57

4060:	learn: 1166.2483762	test: 1541.6691451	best: 1541.6691451 (4060)	total: 9.85s	remaining: 3m 52s
4061:	learn: 1166.2108592	test: 1541.6829921	best: 1541.6691451 (4060)	total: 9.85s	remaining: 3m 52s
4062:	learn: 1166.1470775	test: 1541.6784906	best: 1541.6691451 (4060)	total: 9.85s	remaining: 3m 52s
4063:	learn: 1166.0959721	test: 1541.6484968	best: 1541.6484968 (4063)	total: 9.86s	remaining: 3m 52s
4064:	learn: 1165.9943367	test: 1541.6876892	best: 1541.6484968 (4063)	total: 9.86s	remaining: 3m 52s
4065:	learn: 1165.9398601	test: 1541.6852428	best: 1541.6484968 (4063)	total: 9.86s	remaining: 3m 52s
4066:	learn: 1165.8739931	test: 1541.7315777	best: 1541.6484968 (4063)	total: 9.86s	remaining: 3m 52s
4067:	learn: 1165.7647673	test: 1541.8574136	best: 1541.6484968 (4063)	total: 9.86s	remaining: 3m 52s
4068:	learn: 1165.7197928	test: 1541.8375506	best: 1541.6484968 (4063)	total: 9.87s	remaining: 3m 52s
4069:	learn: 1165.6299033	test: 1541.7722357	best: 1541.6484968 (4063)	total: 9.87

4159:	learn: 1159.1442915	test: 1540.5613794	best: 1540.5613794 (4159)	total: 10s	remaining: 3m 51s
4160:	learn: 1159.0677550	test: 1540.5512927	best: 1540.5512927 (4160)	total: 10s	remaining: 3m 51s
4161:	learn: 1159.0080713	test: 1540.5547202	best: 1540.5512927 (4160)	total: 10.1s	remaining: 3m 51s
4162:	learn: 1158.9555445	test: 1540.5389535	best: 1540.5389535 (4162)	total: 10.1s	remaining: 3m 51s
4163:	learn: 1158.9274251	test: 1540.5248515	best: 1540.5248515 (4163)	total: 10.1s	remaining: 3m 51s
4164:	learn: 1158.8483356	test: 1540.5377783	best: 1540.5248515 (4163)	total: 10.1s	remaining: 3m 51s
4165:	learn: 1158.8109045	test: 1540.5174576	best: 1540.5174576 (4165)	total: 10.1s	remaining: 3m 51s
4166:	learn: 1158.7614936	test: 1540.5558464	best: 1540.5174576 (4165)	total: 10.1s	remaining: 3m 51s
4167:	learn: 1158.6732779	test: 1540.5656393	best: 1540.5174576 (4165)	total: 10.1s	remaining: 3m 51s
4168:	learn: 1158.6232101	test: 1540.5357218	best: 1540.5174576 (4165)	total: 10.1s	re

In [ ]:
X_train_02, X_val_02, y_train_02, y_val_02 = train_test_split(train_02, train_y_02, test_size=0.1, random_state=42)

In [ ]:
params = {
    'iterations' : 100000,
    'learning_rate' : 0.05,
    'loss_function' : 'RMSE',
    'eval_metric': 'RMSE',
    'early_stopping_rounds': 50,
    'task_type': 'GPU',
    'random_state': 42
}

model = CatBoostRegressor(**params)

model.fit(X_train_02, y_train_02, eval_set=(X_val_02, y_val_02))

y_pred_02 = model.predict(test_02)

0:	learn: 514706.9651009	test: 511258.4949674	best: 511258.4949674 (0)	total: 2.91ms	remaining: 4m 51s
1:	learn: 492929.4465004	test: 489854.8034807	best: 489854.8034807 (1)	total: 5.43ms	remaining: 4m 31s
2:	learn: 472597.3987886	test: 469620.0167554	best: 469620.0167554 (2)	total: 7.65ms	remaining: 4m 14s
3:	learn: 453107.5328835	test: 450279.7294894	best: 450279.7294894 (3)	total: 9.98ms	remaining: 4m 9s
4:	learn: 434981.1153507	test: 432376.2937162	best: 432376.2937162 (4)	total: 12.2ms	remaining: 4m 3s
5:	learn: 417556.9777250	test: 415148.9281801	best: 415148.9281801 (5)	total: 14.5ms	remaining: 4m 1s
6:	learn: 401463.7696328	test: 399024.3183991	best: 399024.3183991 (6)	total: 16.6ms	remaining: 3m 57s
7:	learn: 386400.0874481	test: 384269.6821343	best: 384269.6821343 (7)	total: 19ms	remaining: 3m 56s
8:	learn: 371741.0421732	test: 369699.7411654	best: 369699.7411654 (8)	total: 21.1ms	remaining: 3m 54s
9:	learn: 357915.0325143	test: 355980.2949813	best: 355980.2949813 (9)	total: 

118:	learn: 108494.0045258	test: 117216.9547873	best: 117216.9547873 (118)	total: 256ms	remaining: 3m 35s
119:	learn: 108254.7318830	test: 117071.0353939	best: 117071.0353939 (119)	total: 258ms	remaining: 3m 35s
120:	learn: 107668.6681260	test: 116125.1190324	best: 116125.1190324 (120)	total: 261ms	remaining: 3m 35s
121:	learn: 107384.0132256	test: 115636.0539042	best: 115636.0539042 (121)	total: 263ms	remaining: 3m 35s
122:	learn: 107068.7720542	test: 115380.1864691	best: 115380.1864691 (122)	total: 265ms	remaining: 3m 34s
123:	learn: 106657.8977835	test: 115066.5586655	best: 115066.5586655 (123)	total: 267ms	remaining: 3m 34s
124:	learn: 106272.5697989	test: 114744.9872729	best: 114744.9872729 (124)	total: 269ms	remaining: 3m 34s
125:	learn: 105927.1213587	test: 114398.8840066	best: 114398.8840066 (125)	total: 271ms	remaining: 3m 34s
126:	learn: 105217.8295837	test: 113365.6150898	best: 113365.6150898 (126)	total: 273ms	remaining: 3m 34s
127:	learn: 104785.4302620	test: 113010.186449

211:	learn: 82341.5909323	test: 90948.1434532	best: 90948.1434532 (211)	total: 454ms	remaining: 3m 33s
212:	learn: 82055.8685730	test: 90629.5047418	best: 90629.5047418 (212)	total: 456ms	remaining: 3m 33s
213:	learn: 81644.5468658	test: 90302.7729361	best: 90302.7729361 (213)	total: 458ms	remaining: 3m 33s
214:	learn: 81533.9217951	test: 90241.4019191	best: 90241.4019191 (214)	total: 461ms	remaining: 3m 33s
215:	learn: 81366.4802384	test: 90179.2936712	best: 90179.2936712 (215)	total: 463ms	remaining: 3m 33s
216:	learn: 81183.9915326	test: 90035.1514602	best: 90035.1514602 (216)	total: 465ms	remaining: 3m 33s
217:	learn: 80969.8259213	test: 89817.8061519	best: 89817.8061519 (217)	total: 467ms	remaining: 3m 33s
218:	learn: 80551.3324008	test: 89166.6800671	best: 89166.6800671 (218)	total: 469ms	remaining: 3m 33s
219:	learn: 80338.1024523	test: 89010.5554648	best: 89010.5554648 (219)	total: 471ms	remaining: 3m 33s
220:	learn: 80204.0976833	test: 88976.5997663	best: 88976.5997663 (220)	t

302:	learn: 68924.2007562	test: 78137.1472350	best: 78137.1472350 (302)	total: 651ms	remaining: 3m 34s
303:	learn: 68876.2613170	test: 78127.8193241	best: 78127.8193241 (303)	total: 653ms	remaining: 3m 34s
304:	learn: 68857.5524588	test: 78164.7962282	best: 78127.8193241 (303)	total: 656ms	remaining: 3m 34s
305:	learn: 68699.2349962	test: 77966.1061537	best: 77966.1061537 (305)	total: 658ms	remaining: 3m 34s
306:	learn: 68509.5488713	test: 77805.3235661	best: 77805.3235661 (306)	total: 660ms	remaining: 3m 34s
307:	learn: 68384.8742895	test: 77737.8024640	best: 77737.8024640 (307)	total: 662ms	remaining: 3m 34s
308:	learn: 68328.5015569	test: 77714.0248295	best: 77714.0248295 (308)	total: 664ms	remaining: 3m 34s
309:	learn: 68204.2450502	test: 77654.4693198	best: 77654.4693198 (309)	total: 666ms	remaining: 3m 34s
310:	learn: 67936.9651795	test: 77389.3815759	best: 77389.3815759 (310)	total: 668ms	remaining: 3m 34s
311:	learn: 67858.1467910	test: 77351.3709787	best: 77351.3709787 (311)	t

393:	learn: 62072.4438792	test: 72621.0932674	best: 72621.0932674 (393)	total: 849ms	remaining: 3m 34s
394:	learn: 61963.9348150	test: 72529.0660975	best: 72529.0660975 (394)	total: 851ms	remaining: 3m 34s
395:	learn: 61885.2780870	test: 72394.6682480	best: 72394.6682480 (395)	total: 853ms	remaining: 3m 34s
396:	learn: 61771.6962057	test: 72156.9622384	best: 72156.9622384 (396)	total: 855ms	remaining: 3m 34s
397:	learn: 61739.9322618	test: 72135.4352386	best: 72135.4352386 (397)	total: 857ms	remaining: 3m 34s
398:	learn: 61717.8297363	test: 72128.1110827	best: 72128.1110827 (398)	total: 860ms	remaining: 3m 34s
399:	learn: 61620.5624428	test: 72011.5526299	best: 72011.5526299 (399)	total: 862ms	remaining: 3m 34s
400:	learn: 61531.7546461	test: 71985.6604098	best: 71985.6604098 (400)	total: 864ms	remaining: 3m 34s
401:	learn: 61514.0469083	test: 71982.7762451	best: 71982.7762451 (401)	total: 866ms	remaining: 3m 34s
402:	learn: 61440.4872659	test: 71921.6021594	best: 71921.6021594 (402)	t

...

5995:	learn: 18767.4231125	test: 46712.4334167	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
5996:	learn: 18765.1966564	test: 46712.2267328	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
5997:	learn: 18761.4575551	test: 46709.1214044	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
5998:	learn: 18759.6421709	test: 46707.1784023	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
5999:	learn: 18757.4944104	test: 46706.9965000	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
6000:	learn: 18755.3793535	test: 46706.8162508	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
6001:	learn: 18753.2054745	test: 46707.5769310	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
6002:	learn: 18751.9284891	test: 46708.3425596	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
6003:	learn: 18750.0947874	test: 46707.3371527	best: 46701.9161812 (5969)	total: 13.1s	remaining: 3m 24s
6004:	learn: 18747.7683006	test: 46697.6408101	best: 46

6086:	learn: 18598.8050038	test: 46648.5990349	best: 46648.4765107 (6085)	total: 13.3s	remaining: 3m 24s
6087:	learn: 18596.4872826	test: 46649.7133277	best: 46648.4765107 (6085)	total: 13.3s	remaining: 3m 24s
6088:	learn: 18594.0058656	test: 46646.7412690	best: 46646.7412690 (6088)	total: 13.3s	remaining: 3m 24s
6089:	learn: 18592.5351373	test: 46642.3051811	best: 46642.3051811 (6089)	total: 13.3s	remaining: 3m 24s
6090:	learn: 18591.6783129	test: 46642.6545859	best: 46642.3051811 (6089)	total: 13.3s	remaining: 3m 24s
6091:	learn: 18590.7244954	test: 46640.7932713	best: 46640.7932713 (6091)	total: 13.3s	remaining: 3m 24s
6092:	learn: 18588.1010139	test: 46640.9125036	best: 46640.7932713 (6091)	total: 13.3s	remaining: 3m 24s
6093:	learn: 18587.0555812	test: 46641.2685428	best: 46640.7932713 (6091)	total: 13.3s	remaining: 3m 24s
6094:	learn: 18585.8770902	test: 46642.8168679	best: 46640.7932713 (6091)	total: 13.3s	remaining: 3m 24s
6095:	learn: 18583.5124940	test: 46642.3482359	best: 46

6176:	learn: 18443.7411655	test: 46599.8868651	best: 46599.3548172 (6172)	total: 13.5s	remaining: 3m 24s
6177:	learn: 18442.0369212	test: 46598.3934691	best: 46598.3934691 (6177)	total: 13.5s	remaining: 3m 24s
6178:	learn: 18439.7506969	test: 46598.8874056	best: 46598.3934691 (6177)	total: 13.5s	remaining: 3m 24s
6179:	learn: 18437.7602594	test: 46596.9862187	best: 46596.9862187 (6179)	total: 13.5s	remaining: 3m 24s
6180:	learn: 18434.4873939	test: 46593.4140318	best: 46593.4140318 (6180)	total: 13.5s	remaining: 3m 24s
6181:	learn: 18432.8530714	test: 46594.6009233	best: 46593.4140318 (6180)	total: 13.5s	remaining: 3m 24s
6182:	learn: 18430.5889928	test: 46591.3004280	best: 46591.3004280 (6182)	total: 13.5s	remaining: 3m 24s
6183:	learn: 18429.9658799	test: 46590.0255884	best: 46590.0255884 (6183)	total: 13.5s	remaining: 3m 24s
6184:	learn: 18427.7489550	test: 46590.4383816	best: 46590.0255884 (6183)	total: 13.5s	remaining: 3m 24s
6185:	learn: 18426.2109230	test: 46588.2036180	best: 46

6266:	learn: 18281.7821290	test: 46558.5778965	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6267:	learn: 18281.0919695	test: 46558.9760382	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6268:	learn: 18279.6805827	test: 46561.1723926	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6269:	learn: 18278.5545879	test: 46564.9759545	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6270:	learn: 18277.2096891	test: 46563.5328603	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6271:	learn: 18274.7657022	test: 46564.8897019	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6272:	learn: 18272.7986283	test: 46565.7323171	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6273:	learn: 18270.8942915	test: 46562.6918641	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6274:	learn: 18269.4182235	test: 46562.0830862	best: 46557.2772762 (6258)	total: 13.7s	remaining: 3m 24s
6275:	learn: 18268.5603392	test: 46564.4932694	best: 46

6357:	learn: 18139.1172797	test: 46511.1979383	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6358:	learn: 18137.9371306	test: 46511.8937333	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6359:	learn: 18137.0853313	test: 46512.5878574	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6360:	learn: 18135.3607885	test: 46511.4752612	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6361:	learn: 18134.6735760	test: 46510.7213376	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6362:	learn: 18134.0336687	test: 46512.2142273	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6363:	learn: 18131.4378368	test: 46512.3420921	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6364:	learn: 18129.7913453	test: 46513.4048514	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6365:	learn: 18128.1314471	test: 46512.7522537	best: 46510.6266811 (6356)	total: 13.9s	remaining: 3m 24s
6366:	learn: 18126.7147795	test: 46513.8432325	best: 46

In [ ]:
rmse = calculate_rmse(test_y_01, y_pred_01)
r2score = calculate_R2_score(test_y_01, y_pred_01)

In [ ]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)

In [ ]:
rmse = calculate_rmse(test_y_02, y_pred_02)
r2score = calculate_R2_score(test_y_02, y_pred_02)

In [ ]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)